In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

import time
import pandas as pd

def scroll_to_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Define a função para clicar no botão "Ver mais"
def click_ver_mais(driver):
    try:
        ver_mais_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Ver mais')]")
        if ver_mais_button.is_displayed():
            ver_mais_button.click()
            return True
    except:
        return False

# Abrir o navegador Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(r"C:\python312\chromedriver.exe"), options=options) 

# URL do Quinto Andar para casas à venda em São Paulo (exemplo)

driver.get("https://www.quintoandar.com.br/comprar/imovel/campinas-sp-brasil/apartamento")
qnt_total_apt = driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div/main/section[2]/div/div[1]/div/div/div/div[1]/div/p/span').text
qnt_total_apt = int(qnt_total_apt.replace('.',''))
time.sleep(3)

driver.find_element(By.ID, 'SORT_BUTTON').click()
time.sleep(1)
driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/main/section[2]/div/div[1]/div/div/div/div[2]/div/div/div/div/li[4]').click()
time.sleep(3)
menor_valor = driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div/main/section[2]/div/div[2]/div[1]/div/a/div/div/div[2]/div/div[1]/div/div/span[1]/h3').text
menor_valor = int(menor_valor[3:].replace('.',''))
menor_valor = 1900000  # Pulando o inicio 

driver.find_element(By.ID, 'SORT_BUTTON').click()
time.sleep(1)
driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/main/section[2]/div/div[1]/div/div/div/div[2]/div/div/div/div/li[5]').click()
time.sleep(3)
maior_valor = driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div/main/section[2]/div/div[2]/div[1]/div/a/div/div/div[2]/div/div[1]/div/div/span[1]/h3').text
maior_valor = int(maior_valor[3:].replace('.',''))

passo = 200000
intervalo_inferior = menor_valor
intervalo_superior = menor_valor + passo

df_apts = pd.read_csv('links_apartamentos.csv')

while intervalo_superior < maior_valor:
    driver.get(f"https://www.quintoandar.com.br/comprar/imovel/campinas-sp-brasil/apartamento/de-{intervalo_inferior}-a-{intervalo_superior}-venda")
    time.sleep(3)
    driver.find_element(By.ID, 'SORT_BUTTON').click()
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/main/section[2]/div/div[1]/div/div/div/div[2]/div/div/div/div/li[4]').click()
    time.sleep(3)

    qnt_apt = driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div/main/section[2]/div/div[1]/div/div/div/div[1]/div/p/span').text
    qnt_apt = int(qnt_apt.replace('.',''))
    if qnt_apt < 50 and intervalo_superior <= maior_valor: 
        intervalo_superior += passo
        continue
    rows_apt = int(qnt_apt/3)+1
    col_last_line = qnt_apt%3
    print(f'Começando a extrair os links dos apartamentos de {intervalo_inferior} a {intervalo_superior}')

    for i in range (2,rows_apt+1):
        # if len(df_apts)*100/qnt_total_apt >= 100: break
        for j in range(1,4):
            if i == rows_apt and j > col_last_line: break
            try:
                link_apt = driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div/main/section[2]/div/div[{i}]/div[{j}]/div/a').get_attribute("href")
                if link_apt not in df_apts['link'].tolist(): 
                    df_apts = pd.concat([df_apts, pd.DataFrame({'link': [link_apt]})], ignore_index=True)
                    df_apts.to_csv('links_apartamentos.csv', index=False)
                # if len(df_apts)*100/qnt_total_apt >= 100: break
            except:
                print(f"{len(df_apts)*100/qnt_total_apt:.2f}% do total. {(((i-2)*3 + j)*100/qnt_apt):.2f}% desse filtro")
                scroll_to_bottom(driver)
                if click_ver_mais(driver):
                    # Espera um pouco para a nova lista de apartamentos carregar
                    time.sleep(5)
                else:
                    print("Não foi possível encontrar o botão 'Ver mais'.")
                    break  # Sai do loop interno
    intervalo_inferior = intervalo_superior
    intervalo_superior += passo
print(f"Extração concluida, foram encontrados links de {len(df_apts)} apartamentos")



# driver.quit()


Começando a extrair os links dos apartamentos de 1900000 a 5300000
99.78% do total. 18.00% desse filtro
100.12% do total. 50.00% desse filtro
100.40% do total. 74.00% desse filtro
Extração concluida, foram encontrados links de 4067 apartamentos
